In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Optional for nicer plots
sns.set(style="whitegrid")
%matplotlib inline


# Load data
from io import StringIO
csv_file = "csv/bybit_export.csv"
df = pd.read_csv(csv_file, parse_dates=['updatedTime', 'createdTime'])

# Ensure closedPnl is numeric
df['closedPnl'] = pd.to_numeric(df['closedPnl'], errors='coerce')

# Sort by execution time
df.sort_values('updatedTime', inplace=True)

total_trades = len(df)
wins = df[df['closedPnl'] > 0]
losses = df[df['closedPnl'] < 0]
win_rate = len(wins) / total_trades * 100
avg_win = wins['closedPnl'].mean()
avg_loss = losses['closedPnl'].mean()
profit_factor = wins['closedPnl'].sum() / -losses['closedPnl'].sum()
expectancy = (win_rate/100) * avg_win + (1 - win_rate/100) * avg_loss
equity_curve = df['closedPnl'].cumsum()
max_drawdown = (equity_curve.cummax() - equity_curve).max()
final_equity = 1000 + df['closedPnl'].sum()

# Display
print(f"Total trades: {total_trades}")
print(f"Win rate: {win_rate:.2f}%")
print(f"Average win: {avg_win:.2f}")
print(f"Average loss: {avg_loss:.2f}")
print(f"Profit factor: {profit_factor:.2f}")
print(f"Expectancy: {expectancy:.2f} per trade")
print(f"Max drawdown: {max_drawdown:.2f}")
print(f"Final equity (from 1000): {final_equity:.2f}")


In [ ]:
df['cum_pnl'] = df['closedPnl'].cumsum()

# Assume base capital (you can change this)
initial_balance = 1000
df['balance'] = initial_balance + df['cum_pnl']

plt.figure(figsize=(20, 6)) 
plt.plot(df['updatedTime'], df['balance'], label='Cumulative PnL', color='blue')
plt.axhline(initial_balance, color='red', linestyle='--')
plt.title('Equity Curve (Cumulative PnL)')
plt.xlabel('Time')
plt.ylabel('PnL')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# --- Time-Weighted Return (TWR) Calculation ---
df['prev_balance'] = df['balance'].shift(1).fillna(initial_balance)
df['twr_return'] = 1 + (df['closedPnl'] / df['prev_balance'])
twr = df['twr_return'].prod() - 1
print(f"Time-Weighted Return (TWR): {twr:.2%}")

In [ ]:
# --- Drawdown ---
df['rolling_max'] = df['balance'].cummax()
df['drawdown'] = df['balance'] - df['rolling_max']
df['drawdown_pct'] = df['drawdown'] / df['rolling_max']

# Plot drawdown
plt.figure(figsize=(20, 6)) 
plt.fill_between(df['updatedTime'], df['drawdown_pct'], color='red', alpha=0.3)
plt.title("Drawdown (%) Over Time")
plt.ylabel("Drawdown %")
plt.xlabel("Time")
plt.tight_layout()
plt.show()

max_dd = df['drawdown_pct'].min()
print(f"Max Drawdown: {max_dd:.2%}")

In [ ]:
plt.figure(figsize=(20, 6)) 
sns.histplot(df['closedPnl'], bins=30, kde=True, color='purple')
plt.title('Distribution of Closed PnL per Trade')
plt.xlabel('Closed PnL')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()


In [ ]:
# Leverage usage over time
plt.figure(figsize=(20, 6)) 
plt.plot(df['updatedTime'], df['leverage'], label='Leverage', color='purple')
plt.title("Leverage Over Time")
plt.ylabel("Leverage")
plt.xlabel("Time")
plt.tight_layout()
plt.show()

In [ ]:
win_count = (df['closedPnl'] > 0).sum()
loss_count = (df['closedPnl'] <= 0).sum()

plt.figure(figsize=(6, 6))
plt.pie([win_count, loss_count], labels=['Winning Trades', 'Losing Trades'], autopct='%1.1f%%', colors=['green', 'red'])
plt.title('Win vs Loss Trades')
plt.show()


In [ ]:
pnl_by_symbol = df.groupby('symbol')['closedPnl'].count().sort_values()

plt.figure(figsize=(20, 6)) 
pnl_by_symbol.plot(kind='bar', color='orange')
plt.title('Trades by Symbol')
plt.ylabel('Closed PnL Count')
plt.xlabel('Symbol')
plt.xticks(rotation=60)

# Set y-axis ticks with step of 5
max_count = pnl_by_symbol.max()
yticks = np.arange(0, max_count + 5, 5)
plt.yticks(yticks)

plt.tight_layout()
plt.show()


In [ ]:
pnl_by_symbol = df.groupby('symbol')['closedPnl'].sum().sort_values()

plt.figure(figsize=(20, 6)) 
pnl_by_symbol.plot(kind='bar', color='orange')
plt.title('Total PnL by Symbol')
plt.ylabel('Total Closed PnL')
plt.xlabel('Symbol')
plt.xticks(rotation=60)
plt.tight_layout()
plt.show()


In [ ]:
pnl_by_symbol = df.groupby('symbol')['closedPnl'].mean().sort_values()

plt.figure(figsize=(20, 6)) 
pnl_by_symbol.plot(kind='bar', color='orange')
plt.title('Average PnL by Symbol')
plt.ylabel('Average Closed PnL')
plt.xlabel('Symbol')
plt.xticks(rotation=60)
plt.tight_layout()
plt.show()
